## Evaluate Multiple Classifiers

In [1]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from IPython.display import display, HTML
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk, concatenate_datasets
from sklearn.metrics import accuracy_score
import random
from tqdm import tqdm
import numpy as np
import torch
import os

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set Random Seed for Reproducibility

In [2]:
# Set a seed for random module
random.seed(42)

# Set a seed for numpy module
np.random.seed(42)

# Set a seed for torch module
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Define Parameters

In [3]:
#SAMPLING = "random" # "random", "stratified", "clustered", "shared_domain"
SUFFIX = "_extended" #"", "_holdout", "_extended"
SPLITS = ["test", "holdout", "extended"] # "train", "test", "holdout", "extended"
MAX_CONTENT_LENGTH = 384 # 496, 192
OVERLAP = 64
FEATURES = "url_and_content" # "url", "content", "url_and_content"

In [4]:
TOPICS = ["cannabis", "kinder", "energie"]
#TOPICS = ["cannabis"]

In [5]:
MODEL = "deepset/gelectra-large"  # "deepset/gelectra-large"
STRATEGIES = ["random", "stratified", "clustered"]  # , "shared_domain"]

## Helpers

In [6]:
def calc_metrics(labels, preds):
    """
    Calculates the accuracy, precision, recall, and F1 score for the given labels and predictions and returns them in a dictionary.
    """
    
    metrics = {
        'accuracy': accuracy_score(labels, preds),
        'precision': precision_score(labels, preds, average='binary'),
        'recall': recall_score(labels, preds, average='binary'),
        'f1': f1_score(labels, preds, average='binary'),
    }

    return metrics

In [7]:
def sample_random_from_dataset(dataset, n=5, subset='test'):
    """
    Samples n random examples from a specified subset of the dataset.
    """
    n = min(n, len(dataset[subset]))
    random_indices = random.sample(range(len(dataset[subset])), n)
    sampled_dataset = dataset[subset].select(random_indices)
    return sampled_dataset

## Page Level Predictions

In [8]:
from collections import Counter
from sklearn.metrics import precision_recall_curve, auc
import matplotlib.pyplot as plt
import numpy as np

In [9]:
from datasets import DatasetDict, concatenate_datasets, load_from_disk

def load_and_merge_datasets(topic, model_name, sampling, suffix, max_content_length, features, splits):
    """Loads specified splits from disk and merges them into a single dataset. """
    datasets_to_merge = []
    
    for split in splits:
        path = f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{sampling}{suffix}_{max_content_length}_sampling_{model_name.split('/')[1]}_{features}_{split}"
        try:
            dataset = load_from_disk(path)
            if split in dataset:
                datasets_to_merge.append(dataset[split])
            else:
                print(f"Warning: Split '{split}' not found in the loaded dataset from {path}.")
        except Exception as e:
            print(f"Error loading split '{split}' from path '{path}': {e}")
    
    if datasets_to_merge:
        merged_dataset = concatenate_datasets(datasets_to_merge)
        return merged_dataset
    else:
        print("No valid splits provided for merging.")
        return None

def load_dataset(topic, model_name, sampling, suffix, max_content_length, features, split):
    path = f"../../data/tmp/processed_dataset_{topic}_buffed_chunkified_{sampling}{suffix}_{max_content_length}_s_{model_name.split('/')[1]}_{features}_{split}"
    return load_from_disk(path)

def merge_dataset_splits(dataset_dict, splits):
    """Merges specified splits from a DatasetDict into a single dataset."""
    datasets_to_merge = []
    
    for split in splits:
        if split in dataset_dict:
            datasets_to_merge.append(dataset_dict[split])
        else:
            print(f"Warning: Split '{split}' not found in dataset_dict.")
    
    if datasets_to_merge:
        merged_dataset = concatenate_datasets(datasets_to_merge)
        return merged_dataset
    else:
        print("No valid splits provided for merging.")
        return None


from collections import defaultdict
from tqdm import tqdm

def group_dataset_by_url(dataset):
    grouped_dataset = defaultdict(list)
    keys_to_extract = ["text", "domain", "preds", "label", "category", "annotation_type", "lang"]

    for example in dataset:
        url = example.get("view_url") or example.get("domain")
        example_filtered = {key: example[key] for key in keys_to_extract}
        grouped_dataset[url].append(example_filtered)

    return dict(grouped_dataset)


def extract_labels(grouped_dataset):
    labels = []
    for chunks in grouped_dataset.values():
        preds = [chunk["label"] for chunk in chunks]
        labels.append(max(preds))
    return labels

def merge_predictions(grouped_dataset):
    predictions = []
    for chunks in grouped_dataset.values():
        preds = [chunk["preds"] for chunk in chunks]
        predictions.append(max(preds))
    return predictions

def merge_probabilities(grouped_dataset):
    probas = []
    for chunks in grouped_dataset.values():
        probas.append(max(chunk["probas"] for chunk in chunks))
    return probas

def plot_precision_recall_curve(recall, precision, pr_auc, model_name, topic, splits):
    plt.figure()
    plt.plot(recall, precision, label=f'PR AUC = {pr_auc:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title(f'P-R Curve {model_name} - {topic}')
    plt.legend(loc='best')

    filename = f'precision_recall_curve_{model_name.split("/")[1]}_{topic}_{"_".join(splits)}.png'.replace(' ', '_')
    plt.savefig(filename, dpi=300)
    plt.show()

**PR-Curves per Model:**

In [10]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed

# Dictionary to store the processed data
data_dict = defaultdict(dict)

def process_topic(strategy, topic):
    print(f"\n\n###### Loading and processing data for {topic} and strategy {strategy} ###### \n\n")

    try:
        # Load and merge datasets
        dataset = load_and_merge_datasets(topic, MODEL, strategy, SUFFIX, MAX_CONTENT_LENGTH, FEATURES, SPLITS)

        # Group dataset by URL
        grouped_dataset = group_dataset_by_url(dataset)

        # Extract labels and probabilities
        labels = extract_labels(grouped_dataset)
        predictions = merge_predictions(grouped_dataset)

        print(f"Finished processing {strategy} on {topic}.")
        return (strategy, topic, labels, predictions)
    
    except Exception as e:
        print(f"An error occurred while processing {strategy} on {topic}: {e}")
        return (strategy, topic, None, None)

with ProcessPoolExecutor(max_workers=9) as executor:
    future_to_topic = {executor.submit(process_topic, strategy, topic): (strategy, topic) for strategy in STRATEGIES for topic in TOPICS}
    for future in as_completed(future_to_topic):
        strategy, topic = future_to_topic[future]
        labels, predictions = future.result()[2], future.result()[3]
        if labels is not None and predictions is not None:
            data_dict[strategy][topic] = (labels, predictions)

#print(data_dict)




###### Loading and processing data for cannabis and strategy random ###### 



###### Loading and processing data for kinder and strategy random ###### 



###### Loading and processing data for energie and strategy random ###### 



###### Loading and processing data for cannabis and strategy stratified ###### 



###### Loading and processing data for kinder and strategy stratified ###### 



###### Loading and processing data for energie and strategy stratified ###### 



###### Loading and processing data for cannabis and strategy clustered ###### 



###### Loading and processing data for kinder and strategy clustered ###### 



###### Loading and processing data for energie and strategy clustered ###### 










Finished processing stratified on cannabis.
Finished processing random on cannabis.
Finished processing clustered on cannabis.
Finished processing stratified on energie.
Finished processing random on energie.
Finished processing clustered on energie.
Finished processi

In [11]:
data_dict.keys()

dict_keys(['stratified', 'random', 'clustered'])

In [12]:
import numpy as np
from matplotlib import cm

In [17]:
from collections import defaultdict
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
import pandas as pd

# Mapping of topics to legend texts
legend_texts = {
    'kinder': 'Children',
    'energie': 'Energy',
    'cannabis': 'Cannabis'
}

# Desired order of topics for calculation
ordered_topics = ['kinder', 'energie', 'cannabis']

# Dictionary to store evaluation results
evaluation_results = defaultdict(dict)

# Iterate over each model
for strategy in STRATEGIES:
    
    for topic in ordered_topics:
        if topic in data_dict[strategy]:
            labels, predictions = data_dict[strategy][topic]

            # Calculate additional metrics
            predictions_binary = [1 if p >= 0.5 else 0 for p in predictions]
            topic_precision = precision_score(labels, predictions)
            topic_recall = recall_score(labels, predictions)
            topic_accuracy = accuracy_score(labels, predictions)
            topic_f1 = f1_score(labels, predictions)

            # Store metrics in the dictionary
            evaluation_results[strategy][topic] = {
                'accuracy': topic_accuracy,
                'precision': topic_precision,
                'recall': topic_recall,
                'f1': topic_f1
            }

            print(f"Metrics for {MODEL} on {topic}: {evaluation_results[strategy][topic]}")



# Convert the evaluation results to a nested dictionary
evaluation_results = dict(evaluation_results)

# Display evaluation results
print(evaluation_results)


Metrics for deepset/gelectra-large on kinder: {'accuracy': 0.9938920756320374, 'precision': 0.15555555555555556, 'recall': 0.9545454545454546, 'f1': 0.267515923566879}
Metrics for deepset/gelectra-large on energie: {'accuracy': 0.9768026328911937, 'precision': 0.0413564929693962, 'recall': 0.9259259259259259, 'f1': 0.0791765637371338}
Metrics for deepset/gelectra-large on cannabis: {'accuracy': 0.9966075429815928, 'precision': 0.22966507177033493, 'recall': 0.9795918367346939, 'f1': 0.37209302325581395}
Metrics for deepset/gelectra-large on kinder: {'accuracy': 0.9901920863946181, 'precision': 0.10260586319218241, 'recall': 0.9545454545454546, 'f1': 0.18529411764705883}
Metrics for deepset/gelectra-large on energie: {'accuracy': 0.9881918819188192, 'precision': 0.07836990595611286, 'recall': 0.9259259259259259, 'f1': 0.14450867052023122}
Metrics for deepset/gelectra-large on cannabis: {'accuracy': 0.9983037714907964, 'precision': 0.375, 'recall': 0.9795918367346939, 'f1': 0.54237288135

In [18]:
import json

# Define the file path to save the dictionary
file_path = f"eval_results_{FEATURES}_sampling_all_pages.json"

In [19]:
# Save the dictionary to disk as JSON
with open(file_path, "w") as file:
    json.dump(evaluation_results, file)